## Importing Necessary Libraries

In [ ]:
import math, numpy as np, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa

## PMV Calculator: Fanger's Model

In [ ]:
# ---------- PMV (ISO 7730 / ASHRAE 55) ----------
def _svp_pa(t):  # saturation vapor pressure (Pa)
    return 610.5 * math.exp(17.2694 * t / (237.29 + t))

def pmv_iso(ta, tr, v, rh, met=1.2, clo=0.68):
    pa = (rh/100.0)*_svp_pa(ta)/1000.0; icl=0.155*clo; m=met*58.15; mw=m
    fcl = 1.0 if icl<=0 else (1.0+1.29*icl if icl<=0.078 else 1.05+0.645*icl)
    tcl = ta + (35.5-ta)/(3.5*icl+0.1) if icl>0 else ta
    hcf = 12.1*math.sqrt(max(v,1e-8))
    for _ in range(150):
        hr = 4*5.67e-8*((tcl+273.15+tr+273.15)/2)**3
        hc = max(hcf, 2.38*abs(tcl-ta)**0.25)
        tnew=((35.7-0.028*mw)+icl*fcl*(hr*tr+hc*ta))/(1+icl*fcl*(hr+hc))
        if abs(tnew-tcl)<1e-6: break
        tcl=tnew
    r=3.96e-8*fcl*((tcl+273.15)**4-(tr+273.15)**4)
    hc=max(hcf,2.38*abs(tcl-ta)**0.25); c=fcl*hc*(tcl-ta)
    return (0.303*math.exp(-0.036*m)+0.028)*(mw-3.05*(5.733-0.007*mw-pa)
            -0.42*(mw-58.15)-0.0173*m*(5.867-pa)-0.0014*m*(34-ta)-r-c)

## An Example: Calculating PMV

In [ ]:
pmv = pmv_iso(
    ta=28.0, tr=28.1, v=0.05, rh=62.7, met=1.6, clo=0.47
)
print(f"PMV = {pmv:.1f}")  # ~0 is neutral comfort